In [1]:
##in the name of Allah

In [2]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Bidirectional,Input,Conv1D,Flatten,Conv1D
from tensorflow.keras.activations import relu,sigmoid,tanh,linear
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from keras_tuner.tuners import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import optuna


In [3]:
df = pd.read_csv('CBOE_DLY_SPX, 30.csv')

In [4]:
df = df.drop(columns='Volume',axis=1)

In [5]:
df.set_index('time',inplace=True)


In [6]:
df['next_hour_close'] = df['close'].shift(-1)
df['ema5'] = ta.ema(df['close'] , length=5)
df['ema10'] = ta.ema(df['close'] , length=10)
# df['ema20'] = ta.ema(df['close'] , length=20)
# df['ema50'] = ta.ema(df['close'] , length=50)
# df['ema200'] = ta.ema(df['close'] , length=200)
df['rsi'] = ta.rsi(df['close'] , length=14)
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
df = pd.concat([df,macd],axis=1)
df['pivot'] = (df['high'] + df['low'] + df['close']) / 3
df['support_1'] = (2 * df['pivot']) - df['high']
df['support_2'] = df['pivot'] - (df['high'] - df['low'])
df['resistance_1'] = (2 * df['pivot']) - df['low']
df['resistance_2'] = df['pivot'] + (df['high'] - df['low'])
bollinger = ta.bbands(df['close'],length=20,std=2)
df = pd.concat([df,bollinger],axis=1)
stochastic = ta.stoch(df['high'], df['low'], df['close'], k=14, d=3, smooth_k=3)
df = pd.concat([df, stochastic], axis=1)
adx = ta.adx(df['high'], df['low'], df['close'], length=14)
df = pd.concat([df, adx], axis=1)
def fibonacci_retracement(df):
    high = df['high'].max()
    low = df['low'].min()

    diff = high - low

    df['fibonaccilevel1'] = high - 0.236 * diff
    df['fibonaccilevel2'] = high - 0.382 * diff
    df['fibonaccilevel3'] = high - 0.5 * diff
    df['fibonaccilevel4'] = high - 0.618 * diff
    df['fibonaccilevel5'] = high - 0.764 * diff

    return df

# fibonacci_retracement(df)

def stochastic_oscillator(df, period=14, ma_period=3):
    df['L14'] = df['low'].rolling(window=period).min()
    df['H14'] = df['high'].rolling(window=period).max()
    df['%K'] = (df['close'] - df['L14']) * 100 / (df['H14'] - df['L14'])
    df['%D'] = df['%K'].rolling(window=ma_period).mean()
    return df
stochastic_oscillator(df)

# Additional pivot-based levels
df['support_3'] = df['low'] - 2 * (df['high'] - df['pivot'])
df['resistance_3'] = df['high'] + 2 * (df['pivot'] - df['low'])
# Fibonacci levels (common ratios)
fib_range = df['high'] - df['low']
df['fib_382'] = df['close'] + fib_range * 0.382  # Common retracement
df['fib_618'] = df['close'] - fib_range * 0.618  # Golden ratio
# Woodie's variation (uses 2*Close in calculation)
df['woodie_pivot'] = (df['high'] + df['low'] + 2 * df['close']) / 4
df['woodie_s1'] = 2 * df['woodie_pivot'] - df['high']
df['woodie_r1'] = 2 * df['woodie_pivot'] - df['low']
# Camarilla system (uses different multipliers)
df['camarilla_s1'] = df['close'] - (df['high'] - df['low']) * 1.1 / 4
df['camarilla_r1'] = df['close'] + (df['high'] - df['low']) * 1.1 / 4
# Midpoints between key levels
df['mid_support'] = (df['support_1'] + df['support_2']) / 2
df['mid_resistance'] = (df['resistance_1'] + df['resistance_2']) / 2
# Alternative pivot calculation (includes open price)
df['alt_pivot'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
# Moving average of pivot points
df['pivot_ma7'] = df['pivot'].rolling(window=7).mean()
df['pivot_ma21'] = df['pivot'].rolling(window=21).mean()
# Price range calculations
df['daily_range'] = df['high'] - df['low']
df['range_pct'] = (df['daily_range'] / df['close']) * 100

In [7]:
df.dropna(inplace=True)

In [8]:
features = [#'open',
#  'high',
#  'low',
#  'close',
   'ema5',
  'ema10',
 # 'ema20',
 # 'ema50',
 # 'ema200',
  # 'rsi',
 # # 'MACD_12_26_9',
 # # 'MACDh_12_26_9',
 # # 'MACDs_12_26_9',
    #  'pivot',
  # 'support_1',
    'support_2',
  #  'resistance_1']
     'resistance_2']
 # 'BBL_20_2.0',
 # # 'BBM_20_2.0',
 # 'BBU_20_2.0',
 # # 'BBB_20_2.0',
 # # 'BBP_20_2.0',
 # # 'STOCHk_14_3_3',
 # # 'STOCHd_14_3_3',
 # # 'ADX_14',
 # # 'DMP_14',
 # # 'DMN_14',
 # 'L14',
 # 'H14',
 # # '%K',
 # # '%D',
  #  'support_3',
  #  'resistance_3']
        # 'fib_382']
    # 'fib_618']
    #  'woodie_pivot']
#  'woodie_s1',
  #  'woodie_r1']
        #  'camarilla_s1',
        # 'camarilla_r1']
  #  'mid_support',
  # 'mid_resistance']
 # 'alt_pivot',
 # 'pivot_ma7',
 # 'pivot_ma21']
 # # 'daily_range',
 # 'range_pct']
target = ['next_hour_close']

# close              0.999833
# camarilla_s1       0.999827
# camarilla_r1       0.999825
# woodie_pivot       0.999824
# fib_382            0.999819
# pivot              0.999818
# woodie_s1          0.999813
# support_1          0.999810
# woodie_r1          0.999809
# resistance_1       0.999804
# alt_pivot          0.999803
# fib_618            0.999802
# low                0.999782
# mid_support        0.999779
# high               0.999773
# mid_resistance     0.999770
# support_2          0.999737
# resistance_2       0.999722
# open               0.999720
# ema5               0.999698
# support_3          0.999639
# resistance_3       0.999621
# ema10              0.999494
# pivot_ma7          0.999487
# ema20              0.999074
# L14                0.998902
# BBM_20_2.0         0.998797
# H14                0.998785
# pivot_ma21         0.998697
# BBL_20_2.0         0.998270
# BBU_20_2.0         0.998088
# ema50              0.997838
# ema200             0.992243

In [9]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [10]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()
X_scaled = x_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

In [11]:
Sequence_len = 40

def creat_sequences(X,y,sequence_len):
  X_seq = []
  y_seq = []
  for i in range(len(X)-sequence_len):
    X_seq.append(X[i:i+sequence_len])
    y_seq.append(y[i+sequence_len])
  return np.array(X_seq),np.array(y_seq)

In [12]:
X_final,y_final = creat_sequences(X_scaled,y_scaled,Sequence_len)

In [13]:
split_index = int(len(X_final)*0.90)
x_train = X_final[:split_index]
y_train = y_final[:split_index]
x_test = X_final[split_index:]
y_test = y_final[split_index:]

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9539, 40, 4)
(9539, 1)
(1060, 40, 4)
(1060, 1)


In [25]:
def objective(trial):
    # هایپرپارامترهای پایه
    n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 3)
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # تعریف پارامترهای مجزا برای هر لایه LSTM
    lstm_units_list = [
        trial.suggest_int(f"lstm_units_{i}", 32, 512, step=32) 
        for i in range(3)  # حداکثر 3 لایه
    ]
    lstm_dropout = [
        trial.suggest_float(f"lstm_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    lstm_l2 = [
        trial.suggest_float(f"lstm_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # تعریف پارامترهای مجزا برای هر لایه Dense
    dense_units_list = [
        trial.suggest_int(f"dense_units_{i}", 16, 256, step=16)
        for i in range(3)  # حداکثر 3 لایه
    ]
    dense_dropout = [
        trial.suggest_float(f"dense_dropout_{i}", 0.0, 0.5, step=0.1)
        for i in range(3)
    ]
    dense_l2 = [
        trial.suggest_float(f"dense_l2_{i}", 1e-6, 1e-2, log=True)
        for i in range(3)
    ]

    # ساخت مدل
    model = Sequential()

    # اولین لایه LSTM
    model.add(Bidirectional(
        LSTM(
            lstm_units_list[0],
            return_sequences=(n_lstm_layers > 1),
            kernel_regularizer=l2(lstm_l2[0])  # L2 مجزا برای لایه اول
        ),
        input_shape=(Sequence_len, x_train.shape[2])
    ))
    model.add(Dropout(lstm_dropout[0]))  # Dropout مجزا برای لایه اول

    # سایر لایههای LSTM
    for i in range(1, n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)
        model.add(Bidirectional(
            LSTM(
                lstm_units_list[i],
                return_sequences=return_sequences,
                kernel_regularizer=l2(lstm_l2[i])  # L2 مجزا
            )
        ))
        model.add(Dropout(lstm_dropout[i]))  # Dropout مجزا

    # لایههای Dense
    for i in range(n_dense_layers):
        model.add(Dense(
            dense_units_list[i],
            activation='relu',
            kernel_regularizer=l2(dense_l2[i])  # L2 مجزا برای Dense
        ))
        model.add(Dropout(dense_dropout[i]))  # Dropout مجزا

    # لایه خروجی
    model.add(Dense(1, activation='linear'))

    # کامپایل مدل
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    # آموزش مدل
    history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=batch_size,
        epochs=50,
        verbose=1
    )
    
    return min(history.history["val_loss"])

In [26]:
study = optuna.create_study(direction="minimize")  # هدف کاهش loss است
study.optimize(objective, n_trials=100)  # تعداد اجراهای مختلف

[I 2025-03-07 23:06:07,813] A new study created in memory with name: no-name-30e962f4-e9d5-4401-a25e-41c621b1b88c


Epoch 1/50
299/299 [==============================] - 78s 243ms/step - loss: 0.0944 - mae: 0.1174 - val_loss: 0.0797 - val_mae: 0.1131
Epoch 2/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0762 - mae: 0.0942 - val_loss: 0.0633 - val_mae: 0.0651
Epoch 3/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0698 - mae: 0.0896 - val_loss: 0.0616 - val_mae: 0.0816
Epoch 4/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0635 - mae: 0.0832 - val_loss: 0.0567 - val_mae: 0.0767
Epoch 5/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0594 - mae: 0.0821 - val_loss: 0.0483 - val_mae: 0.0449
Epoch 6/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0551 - mae: 0.0782 - val_loss: 0.0449 - val_mae: 0.0426
Epoch 7/50
299/299 [==============================] - 72s 240ms/step - loss: 0.0519 - mae: 0.0763 - val_loss: 0.0525 - val_mae: 0.1046
Epoch 8/50
299/299 [==============================] - 7

[I 2025-03-08 00:05:59,232] Trial 0 finished with value: 0.009840622544288635 and parameters: {'n_lstm_layers': 3, 'n_dense_layers': 1, 'learning_rate': 6.0714128406166276e-05, 'batch_size': 32, 'lstm_units_0': 128, 'lstm_units_1': 480, 'lstm_units_2': 224, 'lstm_dropout_0': 0.0, 'lstm_dropout_1': 0.1, 'lstm_dropout_2': 0.5, 'lstm_l2_0': 1.960826321539831e-05, 'lstm_l2_1': 2.731229496324677e-05, 'lstm_l2_2': 4.4853635483061675e-06, 'dense_units_0': 80, 'dense_units_1': 64, 'dense_units_2': 48, 'dense_dropout_0': 0.2, 'dense_dropout_1': 0.4, 'dense_dropout_2': 0.30000000000000004, 'dense_l2_0': 0.00025438030880953113, 'dense_l2_1': 0.000769851467898253, 'dense_l2_2': 3.32180640082472e-06}. Best is trial 0 with value: 0.009840622544288635.


Epoch 1/50
299/299 [==============================] - 20s 59ms/step - loss: 0.0253 - mae: 0.0656 - val_loss: 0.0095 - val_mae: 0.0613
Epoch 2/50
299/299 [==============================] - 17s 56ms/step - loss: 0.0056 - mae: 0.0354 - val_loss: 0.0104 - val_mae: 0.0787
Epoch 3/50
299/299 [==============================] - 17s 57ms/step - loss: 0.0048 - mae: 0.0361 - val_loss: 0.0065 - val_mae: 0.0581
Epoch 4/50
299/299 [==============================] - 17s 57ms/step - loss: 0.0035 - mae: 0.0310 - val_loss: 0.0033 - val_mae: 0.0306
Epoch 5/50
299/299 [==============================] - 17s 57ms/step - loss: 0.0031 - mae: 0.0300 - val_loss: 0.0035 - val_mae: 0.0377
Epoch 6/50
299/299 [==============================] - 17s 56ms/step - loss: 0.0028 - mae: 0.0304 - val_loss: 0.0038 - val_mae: 0.0450
Epoch 7/50
299/299 [==============================] - 17s 56ms/step - loss: 0.0026 - mae: 0.0298 - val_loss: 0.0034 - val_mae: 0.0410
Epoch 8/50
299/299 [==============================] - 17s 57ms

[I 2025-03-08 00:20:08,305] Trial 1 finished with value: 0.0010321155423298478 and parameters: {'n_lstm_layers': 1, 'n_dense_layers': 2, 'learning_rate': 0.00356499273373347, 'batch_size': 32, 'lstm_units_0': 288, 'lstm_units_1': 64, 'lstm_units_2': 320, 'lstm_dropout_0': 0.1, 'lstm_dropout_1': 0.1, 'lstm_dropout_2': 0.1, 'lstm_l2_0': 1.2452486141084487e-06, 'lstm_l2_1': 0.00031949019745612916, 'lstm_l2_2': 6.434815260703967e-06, 'dense_units_0': 80, 'dense_units_1': 16, 'dense_units_2': 256, 'dense_dropout_0': 0.0, 'dense_dropout_1': 0.0, 'dense_dropout_2': 0.30000000000000004, 'dense_l2_0': 1.3021032377613524e-06, 'dense_l2_1': 0.00028144940289860713, 'dense_l2_2': 1.3373449647439163e-05}. Best is trial 1 with value: 0.0010321155423298478.


...

Epoch 1/50
299/299 [==============================] - 36s 114ms/step - loss: 0.0146 - mae: 0.0747 - val_loss: 0.0100 - val_mae: 0.0848
Epoch 2/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0058 - mae: 0.0537 - val_loss: 0.0027 - val_mae: 0.0337
Epoch 3/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0054 - mae: 0.0508 - val_loss: 0.0032 - val_mae: 0.0402
Epoch 4/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0047 - mae: 0.0476 - val_loss: 0.0039 - val_mae: 0.0493
Epoch 5/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0048 - mae: 0.0478 - val_loss: 0.0020 - val_mae: 0.0263
Epoch 6/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0046 - mae: 0.0465 - val_loss: 0.0068 - val_mae: 0.0730
Epoch 7/50
299/299 [==============================] - 33s 112ms/step - loss: 0.0045 - mae: 0.0461 - val_loss: 0.0040 - val_mae: 0.0518
Epoch 8/50
299/299 [==============================] - 3

[I 2025-03-09 01:08:32,009] Trial 54 finished with value: 0.0010200024116784334 and parameters: {'n_lstm_layers': 1, 'n_dense_layers': 1, 'learning_rate': 0.00031324745515083635, 'batch_size': 32, 'lstm_units_0': 480, 'lstm_units_1': 256, 'lstm_units_2': 224, 'lstm_dropout_0': 0.1, 'lstm_dropout_1': 0.5, 'lstm_dropout_2': 0.30000000000000004, 'lstm_l2_0': 6.8241881860812135e-06, 'lstm_l2_1': 6.411084872207525e-06, 'lstm_l2_2': 0.0016263034502419908, 'dense_units_0': 176, 'dense_units_1': 256, 'dense_units_2': 64, 'dense_dropout_0': 0.2, 'dense_dropout_1': 0.5, 'dense_dropout_2': 0.1, 'dense_l2_0': 1.584380479553368e-06, 'dense_l2_1': 5.915035616258801e-06, 'dense_l2_2': 9.904485448882458e-06}. Best is trial 25 with value: 0.0008266468066722155.


Epoch 1/50
597/597 [==============================] - 56s 90ms/step - loss: 0.0171 - mae: 0.0871 - val_loss: 0.0181 - val_mae: 0.1186
Epoch 2/50
597/597 [==============================] - 53s 88ms/step - loss: 0.0108 - mae: 0.0696 - val_loss: 0.0035 - val_mae: 0.0341
Epoch 3/50
597/597 [==============================] - 53s 89ms/step - loss: 0.0104 - mae: 0.0690 - val_loss: 0.0125 - val_mae: 0.1003
Epoch 4/50
597/597 [==============================] - 53s 89ms/step - loss: 0.0096 - mae: 0.0661 - val_loss: 0.0046 - val_mae: 0.0500
Epoch 5/50
597/597 [==============================] - 53s 89ms/step - loss: 0.0095 - mae: 0.0660 - val_loss: 0.0077 - val_mae: 0.0757
Epoch 6/50
597/597 [==============================] - 53s 88ms/step - loss: 0.0090 - mae: 0.0643 - val_loss: 0.0020 - val_mae: 0.0218
Epoch 7/50
597/597 [==============================] - 53s 89ms/step - loss: 0.0089 - mae: 0.0637 - val_loss: 0.0019 - val_mae: 0.0195
Epoch 8/50
597/597 [==============================] - 53s 88ms

[W 2025-03-09 01:43:52,654] Trial 55 failed with parameters: {'n_lstm_layers': 1, 'n_dense_layers': 1, 'learning_rate': 0.0004081599796309591, 'batch_size': 16, 'lstm_units_0': 448, 'lstm_units_1': 160, 'lstm_units_2': 256, 'lstm_dropout_0': 0.0, 'lstm_dropout_1': 0.5, 'lstm_dropout_2': 0.4, 'lstm_l2_0': 4.1231599983433133e-05, 'lstm_l2_1': 2.236179573784786e-06, 'lstm_l2_2': 0.005112418612694297, 'dense_units_0': 256, 'dense_units_1': 240, 'dense_units_2': 48, 'dense_dropout_0': 0.5, 'dense_dropout_1': 0.5, 'dense_dropout_2': 0.0, 'dense_l2_0': 2.5697493329934824e-06, 'dense_l2_1': 0.00038441431287769925, 'dense_l2_2': 4.4323776428148434e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Mahdimtd\anaconda3\envs\my_env\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Mahdimtd\AppData\Local\Temp\ipykernel_17724\334180372.py", line 82, in objective
    histor

KeyboardInterrupt: 

In [27]:
print("Best hyperparameters: ", study.best_params)


Best hyperparameters:  {'n_lstm_layers': 1, 'n_dense_layers': 1, 'learning_rate': 0.002241032085627275, 'batch_size': 32, 'lstm_units_0': 384, 'lstm_units_1': 160, 'lstm_units_2': 320, 'lstm_dropout_0': 0.0, 'lstm_dropout_1': 0.5, 'lstm_dropout_2': 0.1, 'lstm_l2_0': 5.549602976953121e-05, 'lstm_l2_1': 8.136400649959104e-05, 'lstm_l2_2': 0.009904854205630113, 'dense_units_0': 48, 'dense_units_1': 256, 'dense_units_2': 112, 'dense_dropout_0': 0.0, 'dense_dropout_1': 0.1, 'dense_dropout_2': 0.2, 'dense_l2_0': 1.0993810979042252e-06, 'dense_l2_1': 0.0002639949450923612, 'dense_l2_2': 5.63058462711279e-05}


In [28]:
study = optuna.load_study(
    study_name="your_study_name",
    storage="sqlite:///example.db"
)
print("Best trial until now:", study.best_trial)

KeyError: 'Record does not exist.'